In [19]:
import numpy as np
m=9
n=9
tab=np.zeros([m,n],dtype=int)
obs=np.zeros([m,n],dtype=int)
obs[:,0]=1
obs[0,3:n-2]=1
obs[5:7,3:5]=1
obs[1:3,6]=1
tab[obs==1]=-1

In [20]:
tab

array([[-1,  0,  0, -1, -1, -1, -1,  0,  0],
       [-1,  0,  0,  0,  0,  0, -1,  0,  0],
       [-1,  0,  0,  0,  0,  0, -1,  0,  0],
       [-1,  0,  0,  0,  0,  0,  0,  0,  0],
       [-1,  0,  0,  0,  0,  0,  0,  0,  0],
       [-1,  0,  0, -1, -1,  0,  0,  0,  0],
       [-1,  0,  0, -1, -1,  0,  0,  0,  0],
       [-1,  0,  0,  0,  0,  0,  0,  0,  0],
       [-1,  0,  0,  0,  0,  0,  0,  0,  0]])

In [29]:
constrain=[]
c=[[0,0],2,False,5]#[[i,j],n,(True is vertical ) or (False is herozintal),sum]
constrain.append(c)
c=[[0,3],4,True,12]
constrain.append(c)
c=[[6,4],4,False,16]
constrain.append(c)

In [30]:
constrain

[[[0, 0], 2, False, 5], [[0, 3], 4, True, 12], [[6, 4], 4, False, 16]]

In [89]:
def const_is_sat(table,constrain):
    (i,j)=(constrain[0][0],constrain[0][1])
    n=constrain[1]
    sum=constrain[3]
    if constrain[2]:
        r=table[i+1:i+1+n,j]       
    else:
        r=table[i,j+1:j+1+n]
    if not(np.unique(r[r!=0].size)==(r[r!=0].size)):
        return False
    if sum<np.sum(r):
        return False
    return True

In [90]:
const_is_sat(tab,constrain=constrain[0])

True

In [72]:
(i,j)=(c[0][0],c[0][1])
n=c[1]
sum=c[3]
if c[2]:
    r=tab[i+1:i+1+n,j]       
else:
    r=tab[i,j+1:j+1+n]

In [91]:
np.sum(r)

0